# 1. 환경설정 및 라이브러리 로드

In [ ]:
# !pip install sentence-transformers pandas scikit-learn torch

In [ ]:
# !pip install huggingface_hub

In [ ]:
import json
import os
import random
import logging
from datetime import datetime
from collections import Counter
from dotenv import load_dotenv
from huggingface_hub import login

import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, models, LoggingHandler

# 로깅 설정
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO, handlers=[LoggingHandler()])

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 현재 사용 중인 장치: {device}")


# .env 파일 로드
load_dotenv()

# 환경 변수에서 토큰 가져오기
hf_token = os.getenv("HF_TOKEN")

if hf_token:
    # 토큰을 사용하여 로그인
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
else:
    print("❌ .env 파일에서 HF_TOKEN을 찾을 수 없습니다.")

# 2. 데이터셋 로드 및 분할 (Train,Val,Test)

In [ ]:
# 1. 데이터 불러오기
file_path = './data/final_dataset.json' # 생성하신 파일명 확인

with open(file_path, 'r', encoding='utf-8') as f:
    full_dataset = json.load(f)

# 2. 데이터 분할 (Train 80% : Temp 20%)
train_data, temp_data = train_test_split(
    full_dataset, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

# 3. 나머지 20%를 반으로 나눠 Val 10% : Test 10%
val_data, test_data = train_test_split(
    temp_data, 
    test_size=0.5, 
    random_state=42
)

print(f"📊 데이터 분할 결과")
print(f"· Train: {len(train_data)}건")
print(f"· Validation: {len(val_data)}건")
print(f"· Test: {len(test_data)}건")

In [ ]:
# Train 데이터는 Triplet(Anchor, Positive, Negative) 구조로 생성
train_samples = [
    InputExample(texts=[d['anchor'], d['positive'], d['negative']]) 
    for d in train_data
]

# Validation 데이터는 유사도 측정을 위해 (Anchor, Positive) 쌍으로 생성
val_samples = [
    InputExample(texts=[d['anchor'], d['positive']]) 
    for d in val_data
]

# 학습 데이터 로더 설정 (Batch Size는 메모리에 따라 16~32 조절)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

# 3. 모델 정의 및 손실함수 설정

- Google의 최신 embeddinggemma-300m 모델을 로드합니다.
- 질문(Anchor)과 정답(Positive)은 가깝게, 오답(Negative)은 멀게 만드는 Triplet Loss를 사용하며, 코사인 유사도 기준 마진을 0.5로 설정합니다.

In [14]:
from sentence_transformers import SentenceTransformer, losses, evaluation
from datetime import datetime

# 1. 구글 최신 Gemma 임베딩 모델 로드 (2025년형)
model_name = 'google/embeddinggemma-300m'

print(f"🚀 {model_name} 로딩 중...")
# trust_remote_code=True는 최신 모델 아키텍처를 안전하게 불러오기 위해 필요합니다.
model = SentenceTransformer(model_name, trust_remote_code=True)
print("✅ 구글 최신 모델 로드 성공!")

# 2. Triplet Loss 설정
# Gemma 모델은 코사인 유사도 공간에서 매우 정밀하게 동작하므로 마진 0.5가 적당합니다.
train_loss = losses.TripletLoss(
    model=model, 
    distance_metric=losses.TripletDistanceMetric.COSINE, 
    triplet_margin=0.5
)

# 3. 검증기 설정
# validation 데이터로 학습 중 성능(유사도 점수)을 체크합니다.
val_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    val_samples, 
    name='gemma-benefit-val',
    main_similarity='cosine'
)

🚀 google/embeddinggemma-300m 로딩 중...
2026-02-20 21:29:22,417 - Use pytorch device_name: cuda:0
2026-02-20 21:29:22,418 - Load pretrained SentenceTransformer: google/embeddinggemma-300m


modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

c:\Users\Jeon\anaconda3\envs\sesac-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jeon\.cache\huggingface\hub\models--google--embeddinggemma-300m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

2026-02-20 21:29:55,364 - 14 prompts are loaded, with the keys: ['query', 'document', 'BitextMining', 'Clustering', 'Classification', 'InstructionRetrieval', 'MultilabelClassification', 'PairClassification', 'Reranking', 'Retrieval', 'Retrieval-query', 'Retrieval-document', 'STS', 'Summarization']
✅ 구글 최신 모델 로드 성공!


# 4. 모델 파인튜닝 (Fine-tuning) 실행

- 본격적인 학습을 진행합니다. 학습 속도 향상을 위해 use_amp=True를 사용하며, 5 에포크 동안 검증 점수가 가장 높은 모델을 자동으로 저장합니다.

In [16]:
from sentence_transformers import evaluation
import os

# 1. 저장 경로 및 로그 폴더 생성
output_path = f"./output/gemma_card_finetuned_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_path, exist_ok=True)

print("🚀 구글 Gemma-300m 파인튜닝 시작...")

# 2. 모델 학습 (fit) 실행
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=val_evaluator,           # 100걸음마다 val_samples로 성능 체크
    epochs=5,                          # 8,000건 데이터 기준 5 에포크가 적절
    evaluation_steps=100,              # 성능 검증 주기
    warmup_steps=int(len(train_dataloader) * 0.1), # 학습 초기 안정화 구간
    output_path=output_path,
    save_best_model=True,              # 점수가 가장 높은 모델만 최종 저장
    optimizer_params={'lr': 2e-5},     # Gemma 같은 최신 모델에 권장되는 학습률
    use_amp=True,                      # Mixed Precision 사용 (학습 속도 향상 및 메모리 절약)
    show_progress_bar=True             # 주피터 노트북에서 진행률 확인
)

print(f"✨ 학습 완료! 최적의 모델 저장 경로: {output_path}")

🚀 구글 Gemma-300m 파인튜닝 시작...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Gemma-benefit-val Pearson Cosine,Gemma-benefit-val Spearman Cosine
100,No log,No log,nan,nan


2026-02-20 21:40:23,173 - EmbeddingSimilarityEvaluator: Evaluating the model on the gemma-benefit-val dataset in epoch 0.2570694087403599 after 100 steps:
2026-02-20 21:40:26,339 - Cosine-Similarity:	Pearson: nan	Spearman: nan


c:\Users\Jeon\anaconda3\envs\sesac-env\Lib\site-packages\sentence_transformers\evaluation\EmbeddingSimilarityEvaluator.py:195: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
c:\Users\Jeon\anaconda3\envs\sesac-env\Lib\site-packages\sentence_transformers\evaluation\EmbeddingSimilarityEvaluator.py:196: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)


2026-02-20 21:40:26,343 - Save model to ./output/gemma_card_finetuned_20260220_213426


KeyboardInterrupt: 

## 학습 전

In [15]:
from sentence_transformers import util

# 1. 상위 2개 샘플 가져오기
sample_data = [
    {
        "anchor": "학원비와 편의점에서 자주 결제를 하는데, 이런 곳에서 5% 할인을 받을 수 있는 카드가 있을까요? 전월 실적 조건이 궁금합니다.",
        "positive": "이 카드는 학원, 편의점, 커피전문점에서 5% 할인을 제공하며, 전월 이용실적이 40만원 이상일 경우 적용됩니다."
    },
    {
        "anchor": "매달 200만원 이상 사용하면 1.5% 하나머니가 적립된다고 하는데, 이 혜택이 어떤 가맹점에서 적용되는지 궁금해요. 해외 결제 시에도 적립이 되나요?",
        "positive": "기본적으로 지난달 200만원 이상 사용 시 1.5%의 하나머니가 적립됩니다. 해외 매출 또한 원화 청구 금액에 대해 적립 혜택이 적용됩니다."
    }
]

print("📊 [학습 전 베이스라인 테스트 시작]")
print("-" * 50)

for i, sample in enumerate(sample_data):
    # 구글 모델의 성능 극대화를 위해 'query: ' 접두어 권장
    anchor_emb = model.encode(f"query: {sample['anchor']}", convert_to_tensor=True)
    positive_emb = model.encode(f"passage: {sample['positive']}", convert_to_tensor=True)
    
    # 코사인 유사도 계산
    score = util.cos_sim(anchor_emb, positive_emb).item()
    
    print(f"✅ Sample {i+1}")
    print(f"질문: {sample['anchor'][:30]}...")
    print(f"유사도 점수: {score:.4f}")

📊 [학습 전 베이스라인 테스트 시작]
--------------------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Sample 1
질문: 학원비와 편의점에서 자주 결제를 하는데, 이런 곳에서 ...
유사도 점수: 0.6868


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Sample 2
질문: 매달 200만원 이상 사용하면 1.5% 하나머니가 적립...
유사도 점수: 0.7889


## 학습 후

In [ ]:
# 베스트 모델 로드
best_model = SentenceTransformer(output_path)

print("🔍 [최종 성능 테스트]")
for i, sample in enumerate(test_data[:2]):
    anchor_emb = best_model.encode(f"query: {sample['anchor']}", convert_to_tensor=True)
    pos_emb = best_model.encode(f"passage: {sample['positive']}", convert_to_tensor=True)
    
    score = util.cos_sim(anchor_emb, pos_emb).item()
    print(f"Sample {i+1} 유사도: {score:.4f}")